In [27]:
import bs4 as bs, urllib, pandas as pd, numpy as np

Parse past X years

In [28]:
keyword='medve'
baseurl=u'https://szekelyhon.ro/kereses?op=search&src_words='

In [29]:
start='2002-12'
#end='2018-11-01'
#start='2018-10'
end='2018-12'
dates=[]
datelist = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end), freq='M').tolist()
for date in datelist:
    dates.append(str(date)[:10])

In [30]:
dates[:5]

['2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30']

In [31]:
def extractor(time1,time2):
    time1=dates[i]
    time2=dates[i+1]
    print('Parsing...',time1,'-',time2)
    url=baseurl+keyword+'&src_time1='+time1+'&src_time2='+time2
    html = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(html,'lxml')
    return soup.findAll("div", {"class": "catinner"})

In [ ]:
divs=[]
for i in range(len(dates)-1):
    time1=dates[i]
    time2=dates[i+1]
    divs.append(extractor(time1,time2))

Parsing... 2002-12-31 - 2003-01-31
Parsing... 2003-01-31 - 2003-02-28
Parsing... 2003-02-28 - 2003-03-31
Parsing... 2003-03-31 - 2003-04-30
Parsing... 2003-04-30 - 2003-05-31
Parsing... 2003-05-31 - 2003-06-30
Parsing... 2003-06-30 - 2003-07-31
Parsing... 2003-07-31 - 2003-08-31
Parsing... 2003-08-31 - 2003-09-30
Parsing... 2003-09-30 - 2003-10-31
Parsing... 2003-10-31 - 2003-11-30
Parsing... 2003-11-30 - 2003-12-31
Parsing... 2003-12-31 - 2004-01-31
Parsing... 2004-01-31 - 2004-02-29
Parsing... 2004-02-29 - 2004-03-31
Parsing... 2004-03-31 - 2004-04-30
Parsing... 2004-04-30 - 2004-05-31
Parsing... 2004-05-31 - 2004-06-30
Parsing... 2004-06-30 - 2004-07-31
Parsing... 2004-07-31 - 2004-08-31
Parsing... 2004-08-31 - 2004-09-30
Parsing... 2004-09-30 - 2004-10-31
Parsing... 2004-10-31 - 2004-11-30
Parsing... 2004-11-30 - 2004-12-31
Parsing... 2004-12-31 - 2005-01-31
Parsing... 2005-01-31 - 2005-02-28
Parsing... 2005-02-28 - 2005-03-31
Parsing... 2005-03-31 - 2005-04-30
Parsing... 2005-04-3

In [ ]:
def date_hu_en(i):
    date=i[6:-4]
    if date=='augusztus': m='08'
    elif date=='december': m='12'
    elif date=='február': m='02'
    elif date=='január': m='01'
    elif date=='július': m='07'
    elif date=='június': m='06'
    elif date=='május': m='05'
    elif date=='március': m='03'
    elif date=='november': m='11'
    elif date==u'október': m='10'
    elif date==u'szeptember': m='09'
    elif date==u'április': m='04'
    else: return date
    return i[:4]+'-'+m+'-'+i[-3:-1]

Relevant = Medves cikk-e vagy sem  
Deaths = Halalok szama  
Severity = Sulyossag: 0-mas jellegu hir, 1-latas, 2-allat-tamadas, 3-ember-tamadas

In [ ]:
def text_processor(title,content):
    relevant=0
    severity=0
    deaths=0
    tamadas=[u'támad',u'sebes']
    for i in tamadas:
        if i in title+content:
            relevant=1
            severity=2
    tamadas=[u'halál',u'áldozat',u'ölt ']
    for i in tamadas:
        if i in title+content:
            relevant=1
            severity=3
    tamadas=[u'medve',u'medvék']
    for i in tamadas:
        if i in title.replace(',',' ').replace('.',' ').lower():
            relevant=1
    return relevant,severity,deaths

In [ ]:
hirek=[]
for divgroup in divs:
    for div in divgroup:
        idiv=div.find('div')
        content=div.find('p')
        date=idiv.text[idiv.text.find('201'):idiv.text.find(',')]
        title=div.find('h2').text
        if content==None:
            sdiv=str(div)[::-1]
            content=sdiv[:sdiv.find('>a/<')].replace('\r','').replace('\t','').replace('\n','')[::-1][:-6]
        else: content=content.text
        relevant,severity,deaths=text_processor(title,content)
        hirek.append({'date':date_hu_en(date),
                      'hudate':date,
                      'title':title,
                      'content':content,
                      'link':div.findAll('a')[-1]['href'],
                      'relevant':relevant,
                      'severity':severity,
                      'deaths':deaths,
                      'duplicate':0
                       })

In [ ]:
div

Összes medvés hír

In [23]:
len(hirek)

1

In [24]:
df=pd.DataFrame().from_dict(hirek)
df['date']=pd.to_datetime(df['date'])
df=df.sort_values('date').reset_index(drop=True)

Save to medve Excel. Manual curation

In [25]:
df=df[[ 'date', 'hudate', 'link', 'title','content','relevant', 'severity','deaths']]

In [26]:
df.to_excel('szekelyhon_medve_'+end+'.xlsx')
df.to_excel('szekelyhon_medve_curated_'+end+'.xlsx')